In [1]:
from datetime import datetime
import ccxt

In [ ]:

#Connexion à l'API
kucoin = ccxt.kucoin({
    "apiKey":"62a05a9ccf9fd60001e2ff9e",
    "secret":"578687f6-5420-46d7-9b7a-26a737713cfb",
    "password":"3FiF$4sR6p5Y9fPnSNijL?HaGcd@A7"
})
def getSolde():
    global kucoin
    try:
        for coin in kucoin.fetchBalance()['info']['data'] :
            if coin['currency']=='USDT' :
                return float(coin['balance'])
    except Exception as err:
        raise err
    
def getCurrentPrice(perpSymbol) :
    global kucoin
    try:
        ticker = kucoin.fetchTicker(perpSymbol)
    except BaseException as err:
        print("An error occured", err)
    return float(ticker["ask"])


#Dans le cas où vos clés API seraient mauvaises, cette fonction renverrait une erreur
try :
    test=getSolde();del test
except Exception as err:
    print(f"Problème de connexion à l'API : {err}")
    exit()
print(f"{str(datetime.now()).split('.')[0]} | Connexion à l'API Kucoin réussie.")
date=str(datetime.now()).split('.')[0]
print(f"{date} | Bot de sniping lancé avec {getSolde()} USDT, en attente de nouvelles paires...")

In [50]:
class KucoinAPI:
    def __init__(self) -> None:
        #self._credentials = {
        #    "apiKey":"62fa6d4f37a609000198c9ca",
        #    "secret":"7ab549c3-5fb9-46f6-8f5a-4ba63556102d",
        #    "password":"oHCsD5dDK&HrskYC8a9cYp9N@hgRRg9"
        #}#sandbox
        self._credentials = {
            "apiKey":"62a05a9ccf9fd60001e2ff9e",
            "secret":"578687f6-5420-46d7-9b7a-26a737713cfb",
            "password":"3FiF$4sR6p5Y9fPnSNijL?HaGcd@A7"
        }
        self._session = ccxt.kucoin(self._credentials)
        #self._session.set_sandbox_mode(True)

        
    def authentication_required(fn):
        """Annotation for methods that require auth."""
        def wrapped(self, *args, **kwargs):
            if not self._credentials or "apiKey" not in self._credentials or "secret" not in self._credentials or "password" not in self._credentials: 
                #exit()
                raise Exception(f"You must be authenticated to use this method {fn}") 
            else:
                return fn(self, *args, **kwargs)
        return wrapped
        
    @authentication_required    
    def placeMarketOrder(self,symbol:str,side:str,pct_wallet:float=0.99):
        assert side in ["buy","sell"]
        amount = self.getAmountForTrade(symbol,pct_wallet)
        return self._session.create_market_order(f'{symbol}/USDT', side, amount)
    
    @authentication_required     
    def placeLimitOrder(self):
        pass  
    
    @authentication_required
    def getUSDTBalance(self)->float:
        try:
            print(self._session.fetchBalance({"currency": "usdt"}))
            for coin in self._session.fetchBalance({"currency": "usdt"})['info']['data']:
                print(coin)
                if coin['currency']=='USDT' :
                    return float(coin['balance'])
        except Exception as err:
            raise err
        
    def getCurrentPrice(self,symbol:str)->float:
        try:
            return self._session.fetch_ticker(f'{symbol}/USDT')["ask"]
        except Exception as err:
            raise err
                
    def getAmountForTrade(self, symbol:str,pct_wallet:int=1)->str:
        if self._session.load_markets()[f'{symbol}/USDT']['limits']['amount']['min']<self.getUSDTBalance()*pct_wallet/self.getCurrentPrice(f'{symbol}'):
            amount = self._session.amount_to_precision(f'{symbol}/USDT', self.getUSDTBalance()*pct_wallet/self.getCurrentPrice(f'{symbol}'))
            return amount, float(amount)
        else:
            raise Exception(f"Not enough USDT to trade {symbol}") 
        


In [51]:
kucoin_client = KucoinAPI()

In [55]:
kucoin_client.getCurrentPrice('AVAX3L')

0.002491

In [53]:
kucoin_client.getUSDTBalance()

{'info': {'code': '200000', 'data': [{'id': '61cb714016bee40001cdf96e', 'currency': 'USDT', 'type': 'trade', 'balance': '120.1446638', 'available': '120.1446638', 'holds': '0'}]}, 'timestamp': None, 'datetime': None, 'USDT': {'free': 120.1446638, 'used': 0.0, 'total': 120.1446638}, 'free': {'USDT': 120.1446638}, 'used': {'USDT': 0.0}, 'total': {'USDT': 120.1446638}}
{'id': '61cb714016bee40001cdf96e', 'currency': 'USDT', 'type': 'trade', 'balance': '120.1446638', 'available': '120.1446638', 'holds': '0'}


120.1446638

In [54]:
kucoin_client.getAmountForTrade('BTC',0.05)

{'info': {'code': '200000', 'data': [{'id': '61cb714016bee40001cdf96e', 'currency': 'USDT', 'type': 'trade', 'balance': '120.1446638', 'available': '120.1446638', 'holds': '0'}]}, 'timestamp': None, 'datetime': None, 'USDT': {'free': 120.1446638, 'used': 0.0, 'total': 120.1446638}, 'free': {'USDT': 120.1446638}, 'used': {'USDT': 0.0}, 'total': {'USDT': 120.1446638}}
{'id': '61cb714016bee40001cdf96e', 'currency': 'USDT', 'type': 'trade', 'balance': '120.1446638', 'available': '120.1446638', 'holds': '0'}
{'info': {'code': '200000', 'data': [{'id': '61cb714016bee40001cdf96e', 'currency': 'USDT', 'type': 'trade', 'balance': '120.1446638', 'available': '120.1446638', 'holds': '0'}]}, 'timestamp': None, 'datetime': None, 'USDT': {'free': 120.1446638, 'used': 0.0, 'total': 120.1446638}, 'free': {'USDT': 120.1446638}, 'used': {'USDT': 0.0}, 'total': {'USDT': 120.1446638}}
{'id': '61cb714016bee40001cdf96e', 'currency': 'USDT', 'type': 'trade', 'balance': '120.1446638', 'available': '120.144663

('0.00024769', 0.00024769)

In [58]:
#kucoin_client.placeMarketOrder('BTC','buy',0.05)